# Helper functions

Function to request frames, using the HW trigger.

TrigPeriod = in steps of 10ns.
Example: 50000 = 2kfps

In [4]:
def getFrames(DATADEBUG, numFrames = 2048, TrigPeriod = 50000):

    # damage control
    DATADEBUG.enableDataDebug(True)
    DATADEBUG.cleanData()
    TRIG.StopTriggers()
    
    #sets TriggerRegisters
    TRIG.AcqCountReset()
    TRIG.numberTrigger.set(numFrames)
    TRIG.AutoTrigPeriod.set(TrigPeriod) #in steps of 10ns
    TRIG.AutoRunEn.set(True)
    TRIG.RunTriggerEnable.set(True)
    TRIG.AutoDaqEn.set(True)
    TRIG.DaqTriggerEnable.set(True)

    while (TRIG.AcqCount.get() != numFrames) :
        print("Triggers sent: {}".format(TRIG.AcqCount.get()) , end='\r')
        time.sleep(0.1)

    # stops triggers
    TRIG.StopTriggers()
    time.sleep(0.1)

    while ( numFrames > DATADEBUG.getData().shape[2]) :
        time.sleep(1)
        print("Descrambled {}".format(DATADEBUG.getData().shape[2]), end='\r')
    print("Data Descrambled")
    print(DATADEBUG.getData().shape)

    # Print the status
    DeltaAcqCount = TRIG.AcqCount.get()
    DeltaDaqCount = TRIG.DaqCount.get()
    
    print( f'Delta Aquisition Counter: {DeltaDaqCount}')
    print( f'Number of frames received: {DATADEBUG.getData().shape}' )

    return DATADEBUG.getData()


# Hack to get lanes to behave
Request data and disable lanes which fail

In [5]:
def lanesHackTest(root, PKREG, DATADEBUG):
    PKREG.CountReset()
    for i in range(24):
        print("PKREG.TimeoutCntLane[{}]={}".format(i, PKREG.TimeoutCntLane[i].get()))
    
    # Take data
    PKREG.CountReset()
    imgDesc = getFrames(DATADEBUG, numFrames = 2048)

In [3]:
def lanesHackDisable(root, PKREG, DATADEBUG):   
    TimeoutCntLane = [0] * 24
    disable = PKREG.DisableLane.get()
    for i in range(24):
        print("PKREG.TimeoutCntLane[{}]={}".format(i, PKREG.TimeoutCntLane[i].get()))
        TimeoutCntLane[i] = PKREG.TimeoutCntLane[i].get()
        if(TimeoutCntLane[i]> 20) :
            #disable lane
            disable = disable | 0x1<<i

    PKREG.DisableLane.set(disable)

    #get locked lanes
    root.getLaneLocks()

    #Disabled lanes in ASICS
    for asicIndex in range(4) : 
        root.App.AsicTop.DigAsicStrmRegisters0.enable.set(True)
        print("ASIC{}: {:#x}".format(asicIndex, getattr(root.App.AsicTop, f"DigAsicStrmRegisters{asicIndex}").DisableLane.get()))    